In [1]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline

from lmzintgraf_gp_pref_elicit import dataset, gaussian_process, acquisition_function
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_ccs as utils_ccs
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_data as utils_data
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_experiment as utils_experiment
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_parameters as utils_parameters
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_user as utils_user

In [2]:
map = gpd.read_file("Sidewalk_width_crossings_smaller.geojson") #Read in the map with radius 250m and ~1000 nodes

# Objectives
objective1 = map['length']
objective2 = map['crossing']
objective3 = map['obstacle_free_width']

objectives = ('length', 'crossings')

In [3]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map, approach='primal')
nodes = G.nodes
edges = G.edges

/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [4]:
print(nodes)

[(122245.37633330293, 486126.8581684635), (122254.86602688645, 486129.80052856216), (122264.35426393585, 486132.74411788705), (122273.84081337851, 486135.69101398275), (122283.19823912054, 486138.9678026403), (122284.1816391946, 486139.19731384714), (122287.28805009159, 486137.97386694513), (122291.40829399273, 486140.643590483), (122289.53424561612, 486128.31132216856), (122297.7854183222, 486139.47700177913), (122301.24255780602, 486130.1701925009), (122304.47424118801, 486120.76208520425), (122256.54763350038, 486170.91398380254), (122247.04588395767, 486167.91526857053), (122237.50744934028, 486164.9669819529), (122227.969152229, 486162.0189639581), (122218.43104837791, 486159.0709819753), (122289.24584929895, 486173.25169782597), (122282.43320919861, 486172.4559260068), (122296.3898333376, 486151.2069021054), (122293.35529070727, 486160.70100324217), (122290.30564961619, 486170.1963395323), (122290.12705889999, 486148.4950394649), (122234.28219718198, 486227.4182083386), (122243.0

In [5]:
#Pick random ones or pick manually that make sense - to experiment
S = (122245.37633330293, 486126.8581684635)
T = (122320.31466476223, 486327.5294561802)

In [42]:
def pareto_dominates(a, b):
    """Check if the vector in a Pareto dominates vector b.

    Args:
        a (ndarray): A numpy array.
        b (ndarray): A numpy array.

    Returns:
        bool: Whether vector a dominates vector b.
    """
    a = np.array(a)
    b = np.array(b)
    return np.all(a <= b) and np.any(a < b)

def p_prune(candidates):
    """Create a Pareto coverage set from a set of candidate points.

    References:
        .. [1] Roijers, D. M., & Whiteson, S. (2017). Multi-objective decision making. 34, 129–129.
            https://doi.org/10.2200/S00765ED1V01Y201704AIM034

    Args:
        candidates (Set[Tuple]): A set of vectors.

    Returns:
        Set[Tuple]: A Pareto coverage set.
    """
    pcs = set()
    while candidates:
        vector = candidates.pop()

        for alternative in candidates:
            if pareto_dominates(alternative, vector):
                vector = alternative

        to_remove = set(vector)
        for alternative in candidates:
            if pareto_dominates(vector, alternative):
                to_remove.add(alternative)

        candidates -= to_remove
        pcs.add(vector)
    return pcs

import time
import copy
import numpy as np


def pvi(G, init_pcs, max_iter=1000, epsilon=0.05):
    num_states = len(G.nodes)
    start = time.time()
    dataset = []
    nd_vectors = init_pcs
    nd_vectors_update = copy.deepcopy(nd_vectors)

    last_iter = max_iter - 2  # Denotes the start of the last run.
    is_last = False
    save_iteration = False

    for run in range(max_iter):  # We execute the algorithm for a number of iterations.
        print(f'Value Iteration number: {run}')

        for n, node in enumerate(G.nodes):  # Loop over all states. #node=object; n=number
            print(f'Looping over state {n}')
            for nei, neighbor in enumerate(G.neighbors(node)):
                edge = G[node][neighbor]
                edge_list = [v for k, v in edge.items()]  # Stores only the values of the edges' properties
                for i in objectives:
                  cost = np.array([edge_list[0][i]])

                results = {tuple(cost + value_vec) for value_vec in nd_vectors[n][nei]}  # The set of candidate vectors.
                results = p_prune(results)
                nd_vectors_update[n][nei] = results
        if is_last:
            break
        nd_vectors = copy.deepcopy(nd_vectors_update)  # Else perform a deep copy an go again.

    end = time.time()
    elapsed_seconds = (end - start)
    print("Seconds elapsed: " + str(elapsed_seconds))

    return nd_vectors_update, dataset

init_pcs = [[{tuple(np.full(2, np.inf))} for _ in range(len(G.nodes))] for _ in range(len(G.nodes))]  # Q-set
pvi_result = pvi(G, init_pcs)
print(pvi_result)

Value Iteration number: 0
Looping over state 0
Looping over state 1
Looping over state 2
Looping over state 3
Looping over state 4
Looping over state 5
Looping over state 6
Looping over state 7
Looping over state 8
Looping over state 9
Looping over state 10
Looping over state 11
Looping over state 12
Looping over state 13
Looping over state 14
Looping over state 15
Looping over state 16
Looping over state 17
Looping over state 18
Looping over state 19
Looping over state 20
Looping over state 21
Looping over state 22
Looping over state 23
Looping over state 24
Looping over state 25
Looping over state 26
Looping over state 27
Looping over state 28
Looping over state 29
Looping over state 30
Looping over state 31
Looping over state 32
Looping over state 33
Looping over state 34
Looping over state 35
Looping over state 36
Looping over state 37
Looping over state 38
Looping over state 39
Looping over state 40
Looping over state 41
Looping over state 42
Looping over state 43
Looping over sta

KeyboardInterrupt: 

In [ ]:
# def arg_p_prune(candidates, remove_duplicates=True):
#     """A batched and fast version of the Pareto coverage set algorithm.
#
#         Args:
#             candidates (ndarray): A numpy array of vectors.
#             remove_duplicates (bool, optional): Whether to remove duplicate vectors. Defaults to True.
#
#         Returns:
#             ndarray: A Pareto coverage set.
#         """
#     if len(candidates) == 1:
#         return candidates
#
#     uniques, indcs, invs, counts = np.unique(candidates, return_index=True, return_inverse=True, return_counts=True,
#                                              axis=0)
#
#     res_eq = np.all(candidates[:, None, None] <= candidates, axis=-1).squeeze()
#     res_g = np.all(candidates[:, None, None] < candidates, axis=-1).squeeze()
#     c1 = np.sum(res_eq, axis=-1) == counts[invs]
#     c2 = np.any(~res_g, axis=-1)
#     if remove_duplicates:
#         to_keep = np.zeros(len(candidates), dtype=bool)
#         to_keep[indcs] = 1
#     else:
#         to_keep = np.ones(len(candidates), dtype=bool)
#
#     return np.logical_and(c1, c2) & to_keep
#
#
# def fast_p_prune(candidates, remove_duplicates=True):
#     """A batched and fast version of the Pareto coverage set algorithm.
#
#     Args:
#         candidates (ndarray): A numpy array of vectors.
#         remove_duplicates (bool, optional): Whether to remove duplicate vectors. Defaults to True.
#
#     Returns:
#         ndarray: A Pareto coverage set.
#     """
#     return candidates[arg_p_prune(candidates, remove_duplicates=remove_duplicates)]
#
#
#
# v_n = {}  # Value vector
# objectives = ('length', 'crossing')
# cost = 0
# value_vecs = [{np.inf} for n in G]
#
# for n in G:
#     # v_n[n] = np.inf  # Initialisation of nodes
#
#     if n == T:
#         v_n[n] = 0
#
#     result = []
#
#     for neighbor in G.neighbors(n):
#         edge = G[n][neighbor]
#         edge_list = [v for k, v in edge.items()]  # Stores only the values of the edges' properties
#         value_vecs.append([edge_list[0]['length'], edge_list[0]['crossing']])
#         print(value_vecs)
#         for i in objectives:
#             cost = edge_list[0][i]
#
#         result.append(-(cost + value_vecs))
#     pruned = fast_p_prune([result])
#
#     value_vecs[n] = pruned
#
# print(value_vecs)
#
# #Without prune: 177 v_n
# #With prune: 152 v_n
